# Operational contracts in object detection

Goal: make the detection pipeline inspectable:
candidates (boxes + scores) -> threshold -> IoU -> NMS.

Each step is in its own cell so you can see behavior changes clearly.

## 1. Workspace information

In [ ]:
!uname -a

!free -h || true

!cat /proc/cpuinfo | head -n 20 || true

## 2. Workspace setup (Colab-friendly)

In [ ]:
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
except Exception:
    pass

import os
import glob

my_name = '03_yolo_detection_contract_optional.ipynb'
my_path = glob.glob(os.getcwd() + '/**/' + my_name, recursive=True)

if my_path:
    nb_dir = os.path.dirname(my_path[0])
    os.chdir(os.path.abspath(os.path.join(nb_dir, '..')))

print("Current dir:", os.getcwd())
!ls


## 3. Load demo images

In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

img_dir = Path("data/input/images")
img1 = Image.open(img_dir / "image3.png").convert("RGB")
img2 = Image.open(img_dir / "image3-1.png").convert("RGB")

def show(img, title: str) -> None:
    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.title(title)
    plt.show()

show(img1, "image3.png")
show(img2, "image3-1.png")


## 4. Install YOLO stack (optional)
YOLO adds a heavier dependency. Use if you want a detector with a different bias.

In [ ]:
!pip -q install -r requirements-dev.txt
!pip -q install -r requirements-yolo.txt
!pytest -q

## 5. Run YOLO (ultralytics)
Same idea: run a real detector, then use toy duplicates for the contract.

In [ ]:
from src.yolo_ultralytics_det import run_yolo_ultralytics
from src.viz import draw_boxes

res = run_yolo_ultralytics(img1, model_name="yolov8n.pt", max_detections=20)
res2 = run_yolo_ultralytics(img2, model_name="yolov8n.pt", max_detections=20)

vis = draw_boxes(img1, res.boxes[:10], scores=res.scores[:10], labels=res.labels[:10], color="red")
vis2 = draw_boxes(img2, res2.boxes[:10], scores=res2.scores[:10], labels=res2.labels[:10], color="red")

show(vis, "YOLO detections (top candidates)")
show(vis2, "YOLO detections (top candidates)")

print(len(res.boxes), res.scores[:5], res.labels[:5])
print(len(res2.boxes), res2.scores[:5], res2.labels[:5])


## Contract demo (framework-independent)
Even if a pretrained detector outputs little on synthetic images, the contract is the point.
We create toy duplicates to make thresholding and NMS effects visible.

In [ ]:
from src.boxes import Box
from src.contracts import apply_threshold, nms
from src.viz import draw_boxes

# Using the actual results from the YOLO detector
real_boxes = res.boxes
real_boxes2 = res2.boxes

real_scores = res.scores
real_scores2 = res2.scores

real_labels = res.labels
real_labels2 = res2.labels

raw = draw_boxes(img1, real_boxes, scores=real_scores, labels=real_labels, color="red")
raw2 = draw_boxes(img2, real_boxes2, scores=real_scores2, labels=real_labels2, color="red")

show(raw, "Raw candidates (from YOLO detector)")
show(raw2, "Raw candidates (from YOLO detector)")

In [ ]:
thr = 0.5
b, s, l = apply_threshold(real_boxes, real_scores, real_labels, threshold=thr)
b2, s2, l2 = apply_threshold(real_boxes2, real_scores2, real_labels2, threshold=thr/5)

vis = draw_boxes(img1, b, scores=s, labels=l, color="blue")
vis2 = draw_boxes(img2, b2, scores=s2, labels=l2, color="blue")

show(vis, f"After thresholding (threshold={thr:.2f})")
show(vis2, f"After thresholding (threshold={thr:.2f})")

In [ ]:
keep = nms(b, s, iou_threshold=0.15)
kept_b = [b[i] for i in keep]
kept_s = [s[i] for i in keep]
kept_l = [l[i] for i in keep]

keep2 = nms(b2, s2, iou_threshold=0.15)
kept_b2 = [b2[i] for i in keep2]
kept_s2 = [s2[i] for i in keep2]
kept_l2 = [l2[i] for i in keep2]

vis = draw_boxes(img1, kept_b, scores=kept_s, labels=kept_l, color="green")
vis2 = draw_boxes(img2, kept_b2, scores=kept_s2, labels=kept_l2, color="green")

show(vis, "After NMS (duplicates suppressed)")
show(vis2, "After NMS (duplicates suppressed)")